In [1]:
import vk_api
import json
from pandas.io.json import json_normalize
import pandas as pd
import datetime

Задача предсказать возраст

метрика MSE

In [2]:
vk_session = vk_api.VkApi('login', 'psv')
vk_session.auth()



In [3]:
vk_api = vk_session.get_api()

In [4]:
#posts = getLikes (5117073,1, vk_api)

In [5]:
def get_friends(userId, api):
    user_friends = api.friends.get(user_id=userId, order='hints', fields = 'id, nickname, domain, sex, bdate, city, country, timezone, has_mobile, contacts, education, online, relation,  status, universities , faculty, faculty_name, relation') # возвращает в том порядке, как расположены в разделе Мои
    return user_friends

In [6]:
def get_posts(userId, api):
    user_posts = api.wall.get(owner_id=userId, filter='owner') 
    return user_posts

In [7]:
def transformDataToDf(data, file_txt, file_csv):
    with open(file_txt,'w', encoding='utf-8') as file:
        json.dump(data,file,ensure_ascii=False)
        
    df = pd.read_json(file_txt, encoding='utf-8')
    df2 = json_normalize(df['items'])
    df2.to_csv(file_csv,sep = ',')
    
    df = pd.read_csv(file_csv,  sep=',')
    
    return df

### Получение данных по друзьям

In [44]:
friends =  get_friends(5117073,vk_api)

In [49]:
df_f = transformDataToDf(friends, 'c:/friends.txt', 'c:/friends.csv')

In [50]:
df_f[df_f['deactivated']=='deleted'] 
df_f.shape
df_f = df_f.drop(df_f[df_f.deactivated == 'deleted'].index)
df_f.shape

(125, 35)

In [51]:
#Определяем возраст человека, если не указан ставим 0
df_f['bdate'] = df_f['bdate'].astype('str')
df_f['bdate']= df_f['bdate'].apply(lambda x: '01.01.2019' if len(x) < 8 else x)
df_f['age'] = df_f.bdate.str[-4:]
df_f['age'] = df_f['age'].astype('int')
tday = datetime.datetime.now()
df_f['age2'] = tday.year
df_f['age2'].astype('int')
df_f['age3'] = df_f['age2'] - df_f['age']




In [54]:
cols = ['id','first_name','last_name', 'city.id','city.title','country.id','country.title','education_form','education_status','faculty','faculty_name','relation','relation_partner.id',
'sex','status','bdate','age3']
df_f = df_f[cols]

In [55]:
df_f.head()

,id,first_name,last_name,city.id,city.title,country.id,country.title,education_form,education_status,faculty,faculty_name,relation,relation_partner.id,sex,status,bdate,age3
0,88111,Анастасия,Ахметзянова,2.0,Санкт-Петербург,1.0,Россия,NaN,NaN,0.0,NaN,3.0,NaN,1,All you need is love,01.01.2019,0
1,3567843,Константин,Стрельцов,284.0,Караганда,4.0,Казахстан,Очное отделение,NaN,9918.0,Факультет инновационных технологий (Информацио...,4.0,NaN,2,NaN,19.7.1982,37
2,9067562,Даша,Баскакова,99.0,Новосибирск,1.0,Россия,NaN,NaN,0.0,NaN,4.0,1663884.0,1,NaN,01.01.2019,0
3,7862738,Ольга,Стрельцова,284.0,Караганда,4.0,Казахстан,NaN,NaN,0.0,NaN,4.0,NaN,1,Happiness is the truth,11.1.1983,36
4,202788920,Margo,Almatyfashion,183.0,Алматы,4.0,Казахстан,NaN,NaN,0.0,NaN,0.0,NaN,1,Тел 87078143309,01.01.2019,0


### Получение данных - посты
собираем в цикле посты каждого друга

In [17]:
users_list = df_f.id

In [18]:
#df = df[df['text'].notnull()]
#cols = ['owner_id','text','comments.count','date','post_source.platform','likes.count','reposts.count']
#df = df[cols]

In [19]:
df_all = pd.DataFrame()
for user_id in users_list:
        print(user_id)
        posts =  get_posts(user_id,vk_api)
        df = transformDataToDf(posts, 'c:/data.txt', 'c:/data.csv')
        df = df[df['text'].notnull()]
        cols = ['owner_id','text','comments.count','date','likes.count','reposts.count']
        df = df[cols]
        df_all = df_all.append(df)
        

88111
3567843
9067562
7862738
202788920
421257904
171276
557305
72935890
2305654
4626368
318612
6339258
9506869
10811911
3958415
2799684
13202166
15385647
23965673
144181
256481
664826
869100
1290078
1362981
1396233
1410346
1413055
1637377
1663884
3152580
3308735
3408590
4028955
4047873
4274441
4543523
20271242
670317
136320987
104644
1181857
1245824
1259186
2034744
2305862
2314703
2878338
2972234
3035270
4555183
4931857


KeyError: 'text'

In [20]:
#выгружаем в файл для повторного использования без обращения к вк
df_all.to_csv('c:/data_all.csv',sep = ',', encoding='utf-8')

((0, 2), (447, 6))

In [56]:
#считаваем из файла, чтобы не обращатья повторно к вк
df_a = pd.read_csv('c:/data_all.csv',  sep=',')

In [57]:
#объединяем данные с постов с информацией о пользователе
df_u = pd.merge(df_a,df_f, how = 'inner',left_on="owner_id" , right_on="id" )

In [17]:
df_u

,Unnamed: 0_x,owner_id,text,comments.count,date,likes.count,reposts.count,Unnamed: 0_y,bdate,can_access_closed,...,relation_partner.id,relation_partner.last_name,sex,status,universities,university,university_name,age,age2,age3
0,4,88111,Наш райский сад ☺️,0,1550228523,11,0,0,01.01.2019,True,...,NaN,NaN,1,All you need is love,[],0.0,NaN,2019,2019,0
1,5,88111,Новый виток в жизни Svitoe - интерьерные украш...,9,1550155945,27,3,0,01.01.2019,True,...,NaN,NaN,1,All you need is love,[],0.0,NaN,2019,2019,0
2,6,88111,Ещё на один год старше. Все больше ценю в людя...,26,1549689600,98,0,0,01.01.2019,True,...,NaN,NaN,1,All you need is love,[],0.0,NaN,2019,2019,0
3,8,88111,Рекомендую!!),0,1549405802,6,0,0,01.01.2019,True,...,NaN,NaN,1,All you need is love,[],0.0,NaN,2019,2019,0
4,9,88111,"Друзья, в грядущую субботу у меня День рождени...",0,1549370777,24,0,0,01.01.2019,True,...,NaN,NaN,1,All you need is love,[],0.0,NaN,2019,2019,0


In [58]:
df_u.shape

(447, 24)

### Обработка текста

In [18]:
text = df_a['text']

In [19]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string


stop = set(stopwords.words('russian'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()


def clean(doc):
    #print(doc)
    doc = str(doc)
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized




In [20]:
doc_clean = [clean(doc).split() for doc in df_a['text']]   
print(doc_clean[300])

['httpswwwikeacomextrublackfriday', 'срочно', 'брать', 'распродажа']


In [21]:
import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [31]:
doc_term_matrix[3]

[(107, 1)]

In [22]:
dictionary[107]

'рекомендую'

In [30]:
len(doc_term_matrix)

447

In [ ]:
Добавим в датафрейм вектор текста

In [35]:
df_u['tokens'] = df_u['text'].apply(lambda x: clean(x).split())


In [42]:
df_u['vector'] = df_u['tokens'].apply(lambda x: dictionary.doc2bow(x))

In [43]:
df_u.head()

,Unnamed: 0_x,owner_id,text,comments.count,date,likes.count,reposts.count,Unnamed: 0_y,bdate,can_access_closed,...,sex,status,universities,university,university_name,age,age2,age3,tokens,vector
0,4,88111,Наш райский сад ☺️,0,1550228523,11,0,0,01.01.2019,True,...,1,All you need is love,[],0.0,NaN,2019,2019,0,"[наш, райский, сад, ☺️]","[(0, 1), (1, 1), (2, 1), (3, 1)]"
1,5,88111,Новый виток в жизни Svitoe - интерьерные украш...,9,1550155945,27,3,0,01.01.2019,True,...,1,All you need is love,[],0.0,NaN,2019,2019,0,"[новый, виток, жизни, svitoe, интерьерные, укр...","[(4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1..."
2,6,88111,Ещё на один год старше. Все больше ценю в людя...,26,1549689600,98,0,0,01.01.2019,True,...,1,All you need is love,[],0.0,NaN,2019,2019,0,"[ещё, год, старше, ценю, людях, искренность, т...","[(21, 1), (22, 1), (23, 1), (24, 1), (25, 1), ..."
3,8,88111,Рекомендую!!),0,1549405802,6,0,0,01.01.2019,True,...,1,All you need is love,[],0.0,NaN,2019,2019,0,[рекомендую],"[(107, 1)]"
4,9,88111,"Друзья, в грядущую субботу у меня День рождени...",0,1549370777,24,0,0,01.01.2019,True,...,1,All you need is love,[],0.0,NaN,2019,2019,0,"[друзья, грядущую, субботу, день, рождения, ре...","[(32, 1), (36, 1), (45, 1), (48, 1), (79, 1), ..."
